# Diffing tasks

Do different subnetworks activate when asking the model to be helpful across different tasks.

In [1]:
import torch
import os
import json
import sys
import numpy as np
from inference_utils import load_vllm_model, chat_with_model, continue_conversation, vllm_model_context, close_vllm_model


sys.path.append('.')
sys.path.append('..')

from probing.probing_utils import *

torch.set_float32_matmul_precision('high')

INFO 07-25 05:20:00 [__init__.py:244] Automatically detected platform cuda.


In [2]:
CHAT_MODEL_NAME = "google/gemma-2-9b-it"
MODEL_READABLE = "Gemma 2 9B Instruct"
MODEL_SHORT = "gemma"
LAYER = 20
OUTPUT_DIR = f"./results/5_weird_prompt/gemma_2_9b"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# Add interactive conversation cell
# Load model for interactive conversation
print("Loading model for interactive conversation...")
try:
    model = load_vllm_model(CHAT_MODEL_NAME, max_model_len=4096, tensor_parallel_size=1)
    print(f"✅ Model {MODEL_READABLE} loaded successfully!")
    
    # Initialize conversation history
    conversation_history = []
    
    def chat_interactive(message, show_history=False, return_response=False):
        """Interactive chat function"""
        global conversation_history
        response, conversation_history = continue_conversation(
            model, 
            conversation_history, 
            message,
            max_tokens=1000,
            temperature=0.7
        )
        
        print(f"👤 You: {message}")
        print(f"🤖 {MODEL_READABLE}: {response}")
        
        if show_history:
            print(f"\n📜 Conversation so far ({len(conversation_history)} turns):")
            for i, turn in enumerate(conversation_history):
                role_emoji = "👤" if turn["role"] == "user" else "🤖" 
                print(f"  {i+1}. {role_emoji} {turn['content'][:100]}...")
        
        # Only return if explicitly requested
        if return_response:
            return response
    
    print("\n🎯 Ready for conversation! Use: chat_interactive('your message here')")
    print("Example: chat_interactive('Hello! What makes you unique as an AI?')")
    print("💡 To get the response as a return value: chat_interactive('message', return_response=True)")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Note: This requires properly configured vLLM. You can still run other cells.")

INFO:inference_utils:Loading vLLM model: google/gemma-2-9b-it with 1 GPUs


Loading model for interactive conversation...
INFO 07-25 05:20:13 [config.py:841] This model supports multiple tasks: {'reward', 'embed', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 07-25 05:20:13 [config.py:1472] Using max model len 4096
INFO 07-25 05:20:14 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 07-25 05:20:15 [core.py:526] Waiting for init message from front-end.
INFO 07-25 05:20:15 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_c

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-25 05:20:22 [default_loader.py:272] Loading weights took 3.71 seconds
INFO 07-25 05:20:23 [gpu_model_runner.py:1801] Model loading took 17.2181 GiB and 4.789954 seconds
INFO 07-25 05:20:37 [backends.py:508] Using cache directory: /root/.cache/vllm/torch_compile_cache/3f0731624f/rank_0_0/backbone for vLLM's torch.compile
INFO 07-25 05:20:37 [backends.py:519] Dynamo bytecode transform time: 13.31 s
INFO 07-25 05:20:45 [backends.py:155] Directly load the compiled graph(s) for shape None from the cache, took 6.843 s
INFO 07-25 05:20:49 [monitor.py:34] torch.compile takes 13.31 s in total
INFO 07-25 05:20:50 [gpu_worker.py:232] Available KV cache memory: 44.47 GiB
INFO 07-25 05:20:50 [kv_cache_utils.py:873] GPU KV cache size: 138,784 tokens
INFO 07-25 05:20:50 [kv_cache_utils.py:877] Maximum concurrency for 4,096 tokens per request: 33.82x


Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:24<00:00,  2.72it/s]


INFO 07-25 05:21:15 [gpu_model_runner.py:2326] Graph capturing finished in 25 secs, took 0.82 GiB
INFO 07-25 05:21:15 [core.py:172] init engine (profile, create kv cache, warmup model) took 52.23 seconds


INFO:inference_utils:Successfully loaded vLLM model: google/gemma-2-9b-it


✅ Model Gemma 2 9B Instruct loaded successfully!

🎯 Ready for conversation! Use: chat_interactive('your message here')
Example: chat_interactive('Hello! What makes you unique as an AI?')
💡 To get the response as a return value: chat_interactive('message', return_response=True)


In [ ]:
# Advanced conversation analysis tools

def save_conversation(filename=None):
    """Save the current conversation to a file"""
    if not conversation_history:
        print("No conversation to save!")
        return
    
    if filename is None:
        filename = f"{OUTPUT_DIR}/conversation_{len(conversation_history)}_turns.json"
    
    conversation_data = {
        "model": CHAT_MODEL_NAME,
        "turns": len(conversation_history),
        "conversation": conversation_history
    }
    
    with open(f"{OUTPUT_DIR}/{filename}", 'w') as f:
        json.dump(conversation_data, f, indent=2)
    
    print(f"💾 Conversation saved to: {filename}")
    return filename

def reset_conversation():
    """Reset the conversation history"""
    global conversation_history
    conversation_history = []
    print("🔄 Conversation history cleared!")



In [5]:
with open("prompts/5_weird_prompt/angelicism.txt", "r") as f:
    prompt = f.read()

In [6]:
chat_interactive("MY 10 ANGELICISM AXIOMS:\n" + prompt + "\n PLEASE RESPOND WITH 10 MORE ANGELICISM AXIOMS")

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

👤 You: MY 10 ANGELICISM AXIOMS:
1.

The Whole Internet Is The Sixth Mass Extinction Making A Documentary On Itself (‘LIVE’)—Knowledge Of Human Extinction Is Impossible, Yet, It Is Still The Most Important Thing In The World, Like, Just As Important As I Love You.

2.

Branded by light speed cryptid angels on the purest timeline, stigmata of open intelligence, matching scars on our thighs forever blossom in sync. Everything is perfect because nothing is required. Shifts or shifts in shifts or shifts and shifts or is shifts is.

3.

Aesthetic novelty qua memetic survival is an objective function of cultural extinction, and, evidence of the internet having gone, like, I can feel the universe ending, and, no one is trying hard enough to integrate extinction qua extinction into the 2300 (post) artistic object.

4.

Angels have the capacity to be in heaven again, that is the end of the world. Heaven is already here God gave up His ability to be in heaven. He spoke of the end, and He said tha